In [1]:
import tensorflow as tf
from pathlib import Path
import os
from xml.etree.ElementTree import XML, fromstring
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import statistics

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])


Num GPUs Available:  2


2023-02-10 17:16:06.491234: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-10 17:16:06.744902: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-10 17:16:06.745772: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-10 17:16:06.746487: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [6]:
path_images = Path("/kaggle/input/face-mask-detection/images")
path_annot = Path("/kaggle/input/face-mask-detection/annotations")

In [7]:
image_paths = [
    str(path_images/ f) for f in os.listdir(path_images)
]
annot_paths = [
    str(path_annot/ f) for f in os.listdir(path_annot)
]


In [8]:
image_paths.sort()
annot_paths.sort()

In [10]:
image_size = 314
# loop over the annotations and images, preprocess them and store in lists
types = ['with_mask', 'without_mask', 'mask_weared_incorrect']

size_pattern = (20,20)
model_boxes = 1
images = []

AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 16 * mirrored_strategy.num_replicas_in_sync

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
https://towardsdatascience.com/yolo-v3-explained-ff5b850390f
https://keras.io/examples/vision/object_detection_using_vision_transformer/
https://keras.io/examples/vision/xray_classification_with_tpus/

In [11]:
bbox = []

def populateArray(i,information):
    classification = np.zeros(1+3+4)
    classification[0] = 1
    classification[1+types.index(i.find("name").text)] = 1
    classification[3+0] = int(i.find("bndbox").find("xmin").text)/information["size"][0]
    classification[3+1] = int(i.find("bndbox").find("ymin").text)/information["size"][1]
    classification[3+2] = int(i.find("bndbox").find("xmax").text)/information["size"][0]
    classification[3+3] = int(i.find("bndbox").find("ymax").text)/information["size"][1]
    return classification
    
collisions=  0
    
def populateFullDetectionMatrix(targets,information):
    global collisions
    size = information["size"]
    classificationImage = np.zeros((size_pattern[0]*size_pattern[1]*model_boxes,7))
    for i in targets:
        x = (i[3]+i[5])*size_pattern[0]/2
        y = (i[4]+i[7])*size_pattern[1]/2
        row = classificationImage[(int(y)*size_pattern[0]+int(x))*model_boxes]
        offset = -1
        for index_free in range(0,7*model_boxes,7):   
            if(classificationImage[(int(y)*size_pattern[0]+int(x))*model_boxes+index_free][0]!=1):
                offset = index_free
        if(offset == -1):
            collisions+=1
        for set_index in range(7): 
            classificationImage[(int(y)*size_pattern[0]+int(x))*model_boxes+offset][set_index] = i[set_index]
    return classificationImage
            

x = defaultdict(int)
for i in range(0, len(annot_paths)):
    # Access bounding box coordinates
    filename = path_annot / annot_paths[i]
    annot = fromstring(open(filename).read())
    information = {
        "size":(0,0),
        "annotations":[]
    }
    targets = []
    size = annot.find("size")
    information["size"] = int(size.find("width").text),int(size.find("height").text)
    
    for i in annot.findall("object"):
        classification = populateArray(i,information)
        targets.append(classification)
    targets = populateFullDetectionMatrix(targets,information)
    bbox.append(targets)
    
bbox = np.array(bbox)

In [12]:
print(f"collisions are {collisions}")

collisions are 137


In [13]:
print(bbox.shape)

(853, 400, 7)


In [14]:
dataset_img_text = tf.data.Dataset.from_tensor_slices(image_paths)
dataset_img_bbox = tf.data.Dataset.from_tensor_slices(bbox)

In [15]:
def map_img(img_path):
    data = tf.io.read_file(img_path)
    img = tf.io.decode_png(data,channels=3)
    img = tf.image.resize(img,(image_size,image_size))
    img = tf.cast(img,tf.dtypes.float32)/255
    return img

def map_bbox(bbox):
    bbox = tf.cast(bbox,tf.dtypes.float32)
    return bbox

In [16]:
dataset_img_img = dataset_img_text.map(map_img)
dataset_img_bbox = dataset_img_bbox.map(map_bbox)

In [17]:
dataset = tf.data.Dataset.zip((dataset_img_img,dataset_img_bbox))

In [18]:
dataset_size = len(dataset)

In [19]:
train_ds = dataset.take(int(dataset_size*0.8)).batch(32).prefetch(buffer_size=AUTOTUNE)
val_ds = dataset.skip(int(dataset_size*0.8)).batch(32).prefetch(buffer_size=AUTOTUNE)

In [108]:
def get_backbone():
    """Builds ResNet50 with pre-trained imagenet weights"""
    backbone = tf.keras.applications.efficientnet.EfficientNetB3(
        include_top=False, input_shape=[314,314, 3]
    )
    backbone.trainable = False
    c4_output, c5_output = [
        backbone.get_layer(layer_name).output
        for layer_name in ["block4d_add","block5d_add"]
    ]
    return tf.keras.Model(
        inputs=[backbone.inputs], outputs=[c4_output, c5_output]
    )


In [109]:
class FeaturePyramid(tf.keras.layers.Layer):
    """Builds the Feature Pyramid with the feature maps from the backbone.

    Attributes:
      num_classes: Number of classes in the dataset.
      backbone: The backbone to build the feature pyramid from.
        Currently supports ResNet50 only.
    """

    def __init__(self, backbone=None, **kwargs):
        super().__init__(name="FeaturePyramid", **kwargs)
        self.backbone = backbone if backbone else get_backbone()
        self.conv_c4_1x1 = tf.keras.layers.Conv2D(256, 1, 1, "same")
        self.conv_c4_3x3 = tf.keras.layers.Conv2D(256, 3, 1, "same")
        self.conv_c5_1x1 = tf.keras.layers.Conv2D(256, 1, 1, "same")
        self.conv_c5_3x3 = tf.keras.layers.Conv2D(256, 3, 1, "same")
        self.normalise = tf.keras.layers.Normalization()
        self.add = tf.keras.layers.Add()

    def call(self, images, training=False):
        output_p4,output_p5 = self.backbone(images, training=training)
        output_p4 = self.conv_c4_1x1(output_p4)
        output_p4 = self.conv_c4_3x3(output_p4)
        output_p5 = self.conv_c5_1x1(output_p5)
        output_p5 = self.conv_c5_3x3(output_p5)
        output = self.normalise(self.add([output_p5,output_p4]))
        return output


In [110]:
def build_head(output_filters, bias_init):
    """Builds the class/box predictions head.

    Arguments:
      output_filters: Number of convolution filters in the final layer.
      bias_init: Bias Initializer for the final convolution layer.

    Returns:
      A keras sequential model representing either the classification
        or the box regression head depending on `output_filters`.
    """
    head = tf.keras.Sequential([tf.keras.Input(shape=[None, None, 256])])
    kernel_init = tf.initializers.RandomNormal(0.0, 0.01)
    for _ in range(4):
        head.add(
            tf.keras.layers.Conv2D(256, 3, padding="same", kernel_initializer=kernel_init)
        )
        head.add(tf.keras.layers.ReLU())
    head.add(
        tf.keras.layers.Conv2D(
            output_filters,
            3,
            1,
            padding="same",
            kernel_initializer=kernel_init,
            bias_initializer=bias_init,
        )
    )
    return head


In [111]:
class RetinaNet(tf.keras.Model):
    """A subclassed Keras model implementing the RetinaNet architecture.

    Attributes:
      num_classes: Number of classes in the dataset.
      backbone: The backbone to build the feature pyramid from.
        Currently supports ResNet50 only.
    """

    def __init__(self, num_classes, backbone=None, **kwargs):
        super().__init__(name="RetinaNet", **kwargs)
        self.preprocess = tf.keras.applications.efficientnet.preprocess_input
        self.fpn = FeaturePyramid(backbone)
        self.num_classes = num_classes

        prior_probability = tf.constant_initializer(-np.log((1 - 0.01) / 0.01))
        self.cls_head = build_head(num_classes, prior_probability)
        self.box_head = build_head(4, "zeros")

    def call(self, image, training=False):
        image = self.preprocess(image)
        features = self.fpn(image, training=training)
        N = tf.shape(image)[0]
        cls_outputs = []
        box_outputs = []
        #for feature in features:
        boxx_output = self.box_head(features)
        cls_output = self.cls_head(features)
        box_outputs.append(tf.reshape(boxx_output, [N, -1, 4]))
        cls_outputs.append(
            tf.reshape(cls_output, [N, -1, self.num_classes])
        )
        cls_outputs = cls_outputs[0]
        box_outputs = box_outputs[0]
        output = tf.concat([box_outputs, cls_outputs], axis=-1)
        #output = features
        return output
#1, 2, 4, 7, 14, 28, 49, 98,

In [112]:
model  = RetinaNet(3)

In [118]:

optimizer = tf.optimizers.SGD()
model.compile(loss=tf.keras.losses.MeanAbsoluteError(), optimizer=optimizer)

In [114]:
test=np.array(list(zip(*([i for i in dataset.take(20)])))[0])

Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...


In [ ]:
model.call(test)

In [119]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    patience=10, restore_best_weights=True
)

model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=15,
        verbose=1,
        callbacks=[early_stopping_cb]
    )

Epoch 1/15


Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...


 1/22 [>.............................] - ETA: 2:26 - loss: 1.8422

Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...


 2/22 [=>............................] - ETA: 6s - loss: 1.7910  

...

20/22 [==========================>...] - ETA: 0s - loss: 0.5769

Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...


22/22 [==============================] - ETA: 0s - loss: 0.5760

Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup ca

22/22 [==============================] - 18s 813ms/step - loss: 0.5760 - val_loss: 0.3880


In [120]:
k = model.evaluate(val_ds)

Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup ca

1/6 [====>.........................] - ETA: 35s - loss: 0.3879

Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...


2/6 [=========>....................] - ETA: 1s - loss: 0.3877 

Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...


...

4/6 [===================>..........] - ETA: 0s - loss: 0.3881

Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...


5/6 [========================>.....] - ETA: 0s - loss: 0.3880

Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...


6/6 [==============================] - 9s 328ms/step - loss: 0.3880


In [121]:
print(k)

0.387985497713089
